In [1]:
# get required libraries 
from pulp import *
import pandas as pd

### 15.2.1 Basic LP problem setting 

Based on concept note and lecutres , I have solved the homework assginment usling pulp. 


## Load data ##


In [2]:
# import excel file in dataframe 

df_food = pd.read_excel('diet.xls')

In [3]:
df_food.head()

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
0,Frozen Broccoli,0.16,10 Oz Pkg,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,"Carrots,Raw",0.07,1/2 Cup Shredded,23.7,0.0,0.1,19.2,5.6,1.6,0.6,15471.0,5.1,14.9,0.3
2,"Celery, Raw",0.04,1 Stalk,6.4,0.0,0.1,34.8,1.5,0.7,0.3,53.6,2.8,16.0,0.2
3,Frozen Corn,0.18,1/2 Cup,72.2,0.0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
4,"Lettuce,Iceberg,Raw",0.02,1 Leaf,2.6,0.0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1


In [5]:
df_food.tail(5) # as we can see in excel, the last few rows are related to 
# max and min value and should be excluded 

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
62,"Crm Mshrm Soup,W/Mlk",0.65,1 C (8 Fl Oz),203.4,19.8,13.6,1076.3,15.0,0.5,6.1,153.8,2.2,178.6,0.6
63,"Beanbacn Soup,W/Watr",0.67,1 C (8 Fl Oz),172.0,2.5,5.9,951.3,22.8,8.6,7.9,888.0,1.5,81.0,2.0
64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,NaN,NaN,Minimum daily intake,1500.0,30.0,20.0,800.0,130.0,125.0,60.0,1000.0,400.0,700.0,10.0
66,NaN,NaN,Maximum daily intake,2500.0,240.0,70.0,2000.0,450.0,250.0,100.0,10000.0,5000.0,1500.0,40.0


In [6]:
# max and min if each nutrient 
df_food_lim = df_food[65:]
df_food_lim

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
65,NaN,NaN,Minimum daily intake,1500.0,30.0,20.0,800.0,130.0,125.0,60.0,1000.0,400.0,700.0,10.0
66,NaN,NaN,Maximum daily intake,2500.0,240.0,70.0,2000.0,450.0,250.0,100.0,10000.0,5000.0,1500.0,40.0


In [7]:
df_food = df_food[0:64]

In [8]:
df_food.tail()

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
59,Neweng Clamchwd,0.75,1 C (8 Fl Oz),175.7,10.0,5.0,1864.9,21.8,1.5,10.9,20.1,4.8,82.8,2.8
60,Tomato Soup,0.39,1 C (8 Fl Oz),170.7,0.0,3.8,1744.4,33.2,1.0,4.1,1393.0,133.0,27.6,3.5
61,"New E Clamchwd,W/Mlk",0.99,1 C (8 Fl Oz),163.7,22.3,6.6,992.0,16.6,1.5,9.5,163.7,3.5,186.0,1.5
62,"Crm Mshrm Soup,W/Mlk",0.65,1 C (8 Fl Oz),203.4,19.8,13.6,1076.3,15.0,0.5,6.1,153.8,2.2,178.6,0.6
63,"Beanbacn Soup,W/Watr",0.67,1 C (8 Fl Oz),172.0,2.5,5.9,951.3,22.8,8.6,7.9,888.0,1.5,81.0,2.0


In [9]:
# now we will break this dataframe in dictionaries so as to get exactly what we need to solve linear programming problem

In [50]:
#Since we need each variable and factor seperately , 
# i.e. aij ; ai = each food item and j is nuterient factor of each 

# Create a list of the food items i.e ai
food_items = list(df_food['Foods'])

# Create a dictinary of costs for all food items 
# or ci cost per unit for each food item 
costs = dict(zip(food_items,df_food['Price/ Serving'])) # link it to food item , so its cost of food ecah item

# Create a dictionary of calories for all food items # as aboe its calories of each food item
calories = dict(zip(food_items,df_food['Calories']))

# Create a dictionary of total fat for all food items 
fat = dict(zip(food_items,df_food['Total_Fat g']))

# Create a dictionary of carbohydrates for all food items
carbs = dict(zip(food_items,df_food['Carbohydrates g']))

# for cholestrol, sodium, dietary_fiber, protien, vit A , vit C, Calcium mg, iron mg 

cholest = dict(zip(food_items,df_food['Cholesterol mg']))
sodium = dict(zip(food_items,df_food['Sodium mg']))

fiber = dict(zip(food_items,df_food['Dietary_Fiber g']))

protein = dict(zip(food_items,df_food['Protein g']))
vit_A = dict(zip(food_items,df_food['Vit_A IU']))
vit_C = dict(zip(food_items,df_food['Vit_C IU']))
calcium = dict(zip(food_items,df_food['Calcium mg']))
iron = dict(zip(food_items,df_food['Iron mg']))
# check one of them , full exapanded fooditem iron content 
iron

{'Frozen Broccoli': 2.3,
 'Carrots,Raw': 0.3,
 'Celery, Raw': 0.2,
 'Frozen Corn': 0.3,
 'Lettuce,Iceberg,Raw': 0.1,
 'Peppers, Sweet, Raw': 0.3,
 'Potatoes, Baked': 4.3,
 'Tofu': 6.2,
 'Roasted Chicken': 1.8,
 'Spaghetti W/ Sauce': 2.3,
 'Tomato,Red,Ripe,Raw': 0.6,
 'Apple,Raw,W/Skin': 0.2,
 'Banana': 0.4,
 'Grapes': 0.1,
 'Kiwifruit,Raw,Fresh': 0.3,
 'Oranges': 0.1,
 'Bagels': 1.0,
 'Wheat Bread': 0.7,
 'White Bread': 0.8,
 'Oatmeal Cookies': 0.5,
 'Apple Pie': 0.1,
 'Chocolate Chip Cookies': 0.4,
 'Butter,Regular': 0.0,
 'Cheddar Cheese': 0.2,
 '3.3% Fat,Whole Milk': 0.1,
 '2% Lowfat Milk': 0.1,
 'Skim Milk': 0.1,
 'Poached Eggs': 0.7,
 'Scrambled Eggs': 0.7,
 'Bologna,Turkey': 0.4,
 'Frankfurter, Beef': 0.6,
 'Ham,Sliced,Extralean': 0.2,
 'Kielbasa,Prk': 0.4,
 "Cap'N Crunch": 7.5,
 'Cheerios': 4.5,
 "Corn Flks, Kellogg'S": 1.8,
 "Raisin Brn, Kellg'S": 16.8,
 'Rice Krispies': 1.8,
 'Special K': 4.5,
 'Oatmeal': 1.6,
 'Malt-O-Meal,Choc': 4.7,
 'Pizza W/Pepperoni': 0.9,
 'Taco': 2.4,


In [51]:
# min and max value of each nutrient 
df_food_lim


,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
65,NaN,NaN,Minimum daily intake,1500.0,30.0,20.0,800.0,130.0,125.0,60.0,1000.0,400.0,700.0,10.0
66,NaN,NaN,Maximum daily intake,2500.0,240.0,70.0,2000.0,450.0,250.0,100.0,10000.0,5000.0,1500.0,40.0


In [52]:
prob = LpProblem("Diet_Problem",LpMinimize) # we name and let lpproblem know its Minimization problem




#### adding lp contraints 

#### Now as we had discussed in the concept note, we need to load each aij in the lp problem



In [53]:

# load  amount of food xi and state that it can not be less than zero 
food_vars = LpVariable.dicts("Food",food_items,lowBound=0,cat='Continuous')

# next load optimsation ie, sum of xici , kindly note since we have taken minimization as inital set up, 
#it will automatically make it minimize xici 
prob += lpSum([costs[i]*food_vars[i] for i in food_items])

# now load max and min limit of each aijxi 

prob += lpSum([calories[f] * food_vars[f] for f in food_items]) >= 1500.0 , 'calorieminimum'
prob += lpSum([calories[f] * food_vars[f] for f in food_items]) <= 2500.0 ,'caloriemaximum'



# for cholestrol 
prob += lpSum([cholest[f] * food_vars[f] for f in food_items]) >= 30.0, "cholestMinimum"
prob += lpSum([cholest[f] * food_vars[f] for f in food_items]) <=240.0, "cholestmax"


#  for Fat 
prob += lpSum([fat[f] * food_vars[f] for f in food_items]) >= 20.0, "FatMinimum"
prob += lpSum([fat[f] * food_vars[f] for f in food_items]) <=70.0, "Fatmax"

# for all others 

prob += lpSum([carbs[f] * food_vars[f] for f in food_items]) >= 130.0, "carbsMinimum"
prob += lpSum([carbs[f] * food_vars[f] for f in food_items]) <= 450.0, "carbsMax"


prob += lpSum([sodium[f] * food_vars[f] for f in food_items]) >= 800.0, "sodiumMinimum"
prob += lpSum([sodium[f] * food_vars[f] for f in food_items]) <= 2000.0, "sodiumMax"

prob += lpSum([fiber[f] * food_vars[f] for f in food_items]) >= 125.0, "fibreMinimum"
prob += lpSum([fiber[f] * food_vars[f] for f in food_items]) <= 250.0, "fibreMax"

prob += lpSum([protein[f] * food_vars[f] for f in food_items]) >= 60.0, "proteinMinimum"
prob += lpSum([protein[f] * food_vars[f] for f in food_items]) <= 100.0, "proteinMax"

prob += lpSum([vit_A[f] * food_vars[f] for f in food_items]) >= 1000.0, "vit_AMinimum"
prob += lpSum([vit_A[f] * food_vars[f] for f in food_items]) <= 10000.0, "Vit_AMax"

prob += lpSum([vit_C[f] * food_vars[f] for f in food_items]) >= 400.0, "vit_cMinimum"
prob += lpSum([vit_C[f] * food_vars[f] for f in food_items]) <= 5000.0, "Vit_cMax"

prob += lpSum([calcium[f] * food_vars[f] for f in food_items]) >= 700.0, "calicumMinimum"
prob += lpSum([calcium[f] * food_vars[f] for f in food_items]) <= 1500.0, "calciumMax"

prob += lpSum([iron[f] * food_vars[f] for f in food_items]) >= 10.0, "ironMinimum"
prob += lpSum([iron[f] * food_vars[f] for f in food_items]) <= 40.0, "ironMax"



In [54]:
# solve all equation simultaneously 

prob.solve()

1

In [55]:
# print only where food quantity is above zero 

for v in prob.variables():
    
    if v.varValue > 0:
        print(v.name, "=", v.varValue)



Food_Celery,_Raw = 52.6437
Food_Frozen_Broccoli = 0.259607
Food_Lettuce,Iceberg,Raw = 63.9885
Food_Oranges = 2.29294
Food_Poached_Eggs = 0.141844
Food_Popcorn,Air_Popped = 13.8693


In [56]:
# so as per lp solution the above diet is what is required to be given to all soliders to have minimum cost 
# all nutrients within range 

In [57]:
# how much is minimum cost 

objective = value(prob.objective)
print("The total optimal cost of this balanced diet is: ${}".format(round(objective,2)))

The total cost of this balanced diet is: $4.34


## 15.2 .2 Adding binary constraints 

In [58]:
# now we will add all the additional constraints with binanry variable 

In [59]:
# lets define yi = 0,1 i.e. food is chosen or not 

yi = LpVariable.dicts("Chosen",food_items,0,1,cat='Integer')

# If a food is selected, then a minimum of 1/10 serving must be chosen

for f in food_items:
    prob += food_vars[f]>= yi[f]*0.1 # i.e if any food item is selected , yi =1 then xi >= 0.1
    # however this may result in xi to be infinity yi = 0, so to control it we need another function 
    #it is called maximum value contraints , it basically forces xi = 0 when yi = 0 and also 
    #the value is so high that it does not act as an additional (unintentional) contraints when yi =1
    prob += food_vars[f]<= yi[f]* 100000 # maximum value contraints 
    

    


### Many people dislike celery and frozen broccoli. So at most one, but not both, can be selected


In [60]:
# we need to add below constraint of achieving above condition 
# y(celery) + y(frozen broccoli) <= 1 
# it make sure sum of chosen is maximum 1 , but it can be zero as well 

prob += (yi['Celery, Raw'] + yi['Frozen Broccoli']) <= 1


#### To get day-to-day variety in protein, at least 3 kinds of meat/poultry/fish/eggs must be selected

In [61]:
# we need to first classify which of food items are of these kind and then we should atleast select three of them 

# i am a vegeterian, so may not be able to classify it properly but will still create constriant whcih will work

# the equation would that sum of all chosen food in this category should be always equal or more than 3 


prob += (yi['Roasted Chicken'] + yi['Poached Eggs'] + yi['Scrambled Eggs']+ yi['Bologna,Turkey'] +
        yi['Frankfurter, Beef']+yi['Ham,Sliced,Extralean']+ yi['Kielbasa,Prk']+ yi['Pork']+
        yi['White Tuna in Water']+ yi['Chicknoodl Soup']) >= 3 





In [62]:
# solve all equations with additional condition simultaneously 

prob.solve()

1

In [63]:
# print only where food quantity is above zero 

for v in prob.variables():
    
    if v.varValue > 0:
        print(v.name, "=", v.varValue)


Chosen_Celery,_Raw = 1
Chosen_Kielbasa,Prk = 1
Chosen_Lettuce,Iceberg,Raw = 1
Chosen_Oranges = 1
Chosen_Peanut_Butter = 1
Chosen_Poached_Eggs = 1
Chosen_Popcorn,Air_Popped = 1
Chosen_Scrambled_Eggs = 1
Food_Celery,_Raw = 42.3994
Food_Kielbasa,Prk = 0.1
Food_Lettuce,Iceberg,Raw = 82.8026
Food_Oranges = 3.07718
Food_Peanut_Butter = 1.94297
Food_Poached_Eggs = 0.1
Food_Popcorn,Air_Popped = 13.2233
Food_Scrambled_Eggs = 0.1


In [64]:
# how much is minimum cost 

objective = value(prob.objective)
print("The total optimal cost of this balanced diet is: ${}".format(round(objective,2)))

The total optimal cost of this balanced diet is: $4.51


### the cost is increased slightly but now we have better varitety of food :)